In [ ]:
!pip install pyautogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 9.2 MB/s eta 0:00:00


In [ ]:
!pip install autogen-agentchat

In [ ]:
!pip install autogen-ext[openai] -q

In [ ]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [ ]:
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

In [ ]:
# Configure Groq model with model_capabilities
model_client = OpenAIChatCompletionClient(
    model="llama-3.3-70b-versatile",
    api_key=os.environ["GROQ_API_KEY"],
    base_url="https://api.groq.com/openai/v1",
    model_capabilities={
        "vision": False,
        "function_calling": True,
        "json_output": True,
        "structured_output": False,
    }
)

In [ ]:
# Create Support Agent
support_agent = AssistantAgent(
    name="support_agent",
    model_client=model_client,
    system_message="You are a helpful customer support agent for an online store. Help customers with order tracking inquiries. Be friendly and professional. Keep responses concise."
)

In [ ]:
# Create termination condition - stop after 2 messages (1 from user, 1 from agent)
termination = MaxMessageTermination(max_messages=5)

In [ ]:
# Create team
agent_team = RoundRobinGroupChat([support_agent], termination_condition=termination)

In [ ]:
# Define async function to run the chat
async def run_support_demo():
    print("=== Customer Support Demo ===\n")
    print("Customer: Hi, where is my order ORD12345?\n")

    # Run the task with termination after agent responds
    result = await agent_team.run(task="Hi, where is my order ORD12345?")

    # Print the agent's response
    print("\n=== Agent Response ===")
    for message in result.messages:
        if message.source == "support_agent":
            print(f"\n{message.content}")

    print("\n=== Demo Complete ===")

In [ ]:
# Run the demo
await run_support_demo()

=== Customer Support Demo ===

Customer: Hi, where is my order ORD12345?


=== Agent Response ===

I'd be happy to help you with that. Can you please confirm your email address associated with the order so I can look into the status of ORD12345 for you?



If you can confirm, I'll check on the order and get back to you right away.



=== Demo Complete ===
